# Importación de librerias

In [2]:
#Importacion de librerias

import random    #Libreria para poder trabajar con valores aletarorios
from time import sleep  #Importacion de método para detener el tiempo de ejecución
from selenium import webdriver  #Importacion de libreria selenium para scraping   
from webdriver_manager.chrome import ChromeDriverManager #Importacion para navegación en google chrome  
from bs4 import BeautifulSoup #Importacion de BeautifulSoup para scraping
import requests #Importacion para poder obtener datos de una pagina web en especifico
import pandas as pd  #Importacion de libreria Pandas 
from datetime import datetime #Importacion de datetime para la obtención de la fecha de la extracción
import math


driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\ramir\.wdm\drivers\chromedriver\win32\99.0.4844.51]
C:\Users\ramir\AppData\Local\Temp\ipykernel_3956\3490631331.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


# Extraccion de datos en Hites

In [3]:
#URL modificada que contiene todos los equipos presentes en la tienda Hites para que se visualize en una sola pagina
url = "https://www.hites.com/tecnologia/computacion/notebook/?start=0&sz=100"

#Definición base de beautifulSoup donde se trabaja el codigo presente 
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data,"html.parser")


bloque = soup.find_all("div",{"class":"col-6 col-lg-4 px-0"})

In [4]:
#Obtención de Urls de productos de hites
url_notebooks = []
driver.get(url)
notebooks = driver.find_elements_by_xpath('//div[@class="col-6 col-lg-4 px-0"]')
for i in notebooks:
    link = i.find_element_by_tag_name('a').get_attribute('href')
    url_notebooks.append(link)

C:\Users\ramir\AppData\Local\Temp\ipykernel_3956\228405683.py:4: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  notebooks = driver.find_elements_by_xpath('//div[@class="col-6 col-lg-4 px-0"]')
C:\Users\ramir\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\remote\webelement.py:341: UserWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  warnings.warn("find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead")


In [5]:
#Creación de listas auxiliares para el ordenamiento y almacenamiento de datos
nombres = []
precioaux = []
precio = []
codigos = []
modelo = []
encontrarIndiceModelo = []
encontrarindice = []
caracteristica = []
almacenamientoaux = []
almacenamiento = []
procesadoraux = []
procesador = []
ramaux = []
ram = []
precios = []

#Ciclo iterativo para recorrer todos los elementos dentro de la pagina
for i in bloque:
    #Obtencion del nombre del producto
    nombre = i.find("a",{"class":"link product-name--bundle"})
    #Obtencion de las caracteristicas
    precio = precios = i.find("span",{"class":"value"})
    
    precioaux.append(precio)
    nombres.append(nombre)

###Obtener cada elemento de almacenamiento, memoria ram y procesador definiendo un valor auxiliar 
###para almacenamiento en caso de no encontrarse en la pagina
for i in url_notebooks:       
    response = requests.get(i)
    data = response.text
    soup = BeautifulSoup(data,"html.parser")
    
    caracteristica = soup.find_all("div", {"class":"attribute-values"})
    codigo = soup.find("span", {"class":"product-sku"})
    codigos.append(codigo)

    if(len(caracteristica) != 3):
        almacenamientoaux.append("null")
        ramaux.append("null")
        procesadoraux.append("null")
        
    else:        
        almacenamientoaux.append(caracteristica[0])
        ramaux.append(caracteristica[1])
        procesadoraux.append(caracteristica[2])

#Insertar elementos de procesador y memoria ram en las listas como texto, en el caso de almacenamiento como str
for i in range(len(almacenamientoaux)):
    if(type(almacenamientoaux[i]==str)):
        almacenamiento.append(almacenamientoaux[i])
        procesador.append(procesadoraux[i])
        ram.append(ramaux[i])
        
    else:
        almacenamiento.append(almacenamientoaux[i].text)
        procesador.append(procesadoraux[i].text)
        ram.append(ramaux[i].text)  

#Modificar los elementos de almacenamiento a formato text
for i in range(len(almacenamiento)):
    try:
        almacenamiento[i] = almacenamiento[i].text
        
    except:
        almacenamiento[i] = almacenamiento[i]
    

#Modicar valor auxiliar de almacenamiento en caso de que no se encontrará en la pagina          
for i in range(len(almacenamiento)):
    if ('RAM' in almacenamiento[i])==True:
        almacenamiento[i]="Almacenamiento no especificado"
        
for i in range(len(precioaux)):
    precioaux[i] = precioaux[i].text
    try:
        ram[i] = ram[i].text
    except:
        ram[i] = "NULL"
    codigos[i] = codigos[i].text
    try:
        nombres[i] = nombres[i].text
    except:
        nombres[i] = nombres[i]
    try:
        procesador[i] = procesador[i].text
    except:
        procesador[i] = procesador[i]
    
          
#Encontrar el indice del primer slash para poder separar el nombre original sin los componentes
for i in range(len(nombres)):
    encontrarindice.append(nombres[i].find("/"))

#Dejar solo el nombre de los notebooks dentro de la lista
for i in range(len(nombres)):
    nombres[i] = nombres[i][0:encontrarindice[i]-1]
    

#Eliminación de caracteres especiales
for i in range(len(precioaux)):
    precioaux[i] = precioaux[i].replace("\n","")
    precioaux[i] = precioaux[i].replace("$","")
    precioaux[i] = precioaux[i].replace(".","")
    precioaux[i] = precioaux[i].replace("Normal","")
    precioaux[i] = precioaux[i].replace("Oferta","")    
    
for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].replace("Almacenamiento:", "")
    almacenamiento[i] = almacenamiento[i].replace("\n","")
    
for i in range(len(procesador)):
    procesador[i] = procesador[i].replace("Procesador:", "")
    procesador[i] = procesador[i].replace("\n","")
    
for i in range(len(ram)):
    ram[i] = ram[i].replace("Memoria RAM:", "")
    ram[i] = ram[i].replace("\n", "")

for i in range(len(codigos)):
    codigos[i] = codigos[i].replace("Código: ", "")
    codigos[i] = codigos[i].replace("\n","")  
    
#Encontrar el indice para el modelo

for i in range(len(nombres)):
    encontrarIndiceModelo.append(nombres[i].find(" "))
    
for i in range(len(nombres)):
    modelo.append(nombres[i][encontrarIndiceModelo[i]:len(nombres[i])])
    
for i in range(len(modelo)):
    modelo[i] = modelo[i][1:len(nombres)]

KeyboardInterrupt: 

In [ ]:
#Lectura del archivo excel que contiene los datos obtenidos de los computadores
#y almacenamiento en un dataframe
df = pd.read_excel("NotebooksOficial.xlsx")
df = df.drop(["Unnamed: 0"], axis=1)
try:
    df = df.drop(["Unnamed: 0.1.1"], axis=1)
    x = datetime.today().strftime('%Y-%m-%d')
    tienda = "Hites"
except:
    x = datetime.today().strftime('%Y-%m-%d')
    tienda = "Hites"



In [ ]:
#Ingreso de valores obtenidos en la pagina de hites en el dataframe importado
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombres[i], 
                    'Codigo':codigos[i], 
                    'Modelo':modelo[i] ,
                    'Almacenamiento':almacenamiento[i],
                    'Procesador':procesador[i], 
                    'Memoria Ram':ram[i], 
                    'Precio':precioaux[i], 
                    'Fecha':x, 
                    'Tienda':tienda, 
                    'Url': url_notebooks[i] },ignore_index=True)

# Extraccion de datos en Pc Factory

In [ ]:
#Urls para obtener los datos desde el sitio web de Pc Factory
url_base=[]
url_notebooks=[]
url1 = "https://www.pcfactory.cl/notebooks?categoria=735&papa=636"
url2 = "https://www.pcfactory.cl/notebooks?categoria=735&papa=636&pagina=2"
url3 = "https://www.pcfactory.cl/notebooks?categoria=735&papa=636&pagina=3"

url_base.append(url1)
url_base.append(url2)
url_base.append(url3)


for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="product "]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)


In [ ]:
#Definición de listas auxiliares
marca = []
nombres = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigos = []
ram = []
precios = []

#Recorrer los equipos a partir de las URL
for i in url_notebooks:
    maux = 0
    moaux = 0
    almaux = 0
    proaux = 0
    ramaux = 0
    try: 
        driver.get(i)
        bloque_codigos = driver.find_elements_by_xpath('//p[@class="link color-dark"]')

        codigo = bloque_codigos[1]
        codigos.append(codigo.text)

        precio = driver.find_element_by_xpath('//div[@class="price-xl color-primary-1"]').text
        precios.append(precio)       
        
        caracteristicas = driver.find_elements_by_xpath("//li")

        for i in range(len(caracteristicas)):
            caracteristicas[i] = caracteristicas[i].text

        for i in caracteristicas:
            if(i.find("Marca")!=-1):
                maux = maux + 1
                nombre_marca = i
            if(i.find("Modelo")!=-1):
                moaux = moaux + 1
                nombre_modelo = i
            if(i.find("Almacenamiento")!=-1):
                almaux = almaux + 1
                nombre_almacenamiento = i
            if(i.find("RAM:")!=-1 or i.find("Memoria:")!=-1 or i.find("Memoria Total:")!=-1):
                ramaux = ramaux + 1
                nombre_ram = i
            if(i.find("Procesador:")!=-1):
                proaux = proaux + 1
                nombre_procesador = i

        if(maux >= 1):
            marca.append(nombre_marca)
        elif(maux == 0):
            marca.append("Marca no encontrada")
        if(moaux >= 1):
            modelo.append(nombre_modelo)
        elif(moaux == 0):
            modelo.append("Modelo no encontrado")
        if(almaux >= 1):
            almacenamiento.append(nombre_almacenamiento)
        elif (almaux == 0 ):
            almacenamiento.append("Almacenamiento no encontrado")
        if(ramaux >= 1):
            ram.append(nombre_ram)
        elif(ramaux == 0):
            ram.append("Memoria ram no encontrada")
        if(proaux >= 1):
            procesador.append(nombre_procesador)
        elif(proaux == 0):
            procesador.append("Procesador no encontrado")

        sleep(random.uniform(2,4))
    except:
        x = 2

#Eliminación de caracteres especiales y filtro de caracteres extra
for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].replace("\xa0", " ")
    almacenamiento[i] = almacenamiento[i].replace("Almacenamiento:", "")
    almacenamiento[i] = almacenamiento[i].lstrip()

for i in range(len(marca)):
    marca[i] = marca[i].replace("\xa0"," ")
    marca[i] = marca[i].replace("Marca:","")
    marca[i] = marca[i].lstrip()

for i in range(len(modelo)):
    modelo[i] = modelo[i].replace("\xa0", " ")
    modelo[i] = modelo[i].replace("Modelo:", "")
    modelo[i] = modelo[i].lstrip()

for i in range(len(ram)):
    ram[i] = ram[i].replace("\xa0", " ")
    ram[i] = ram[i].replace("Memoria RAM:", "")
    ram[i] = ram[i].replace("Memoria:", "")
    ram[i] = ram[i].replace("Memoria Total:", "")
    ram[i] = ram[i].lstrip()

for i in range(len(procesador)):
    procesador[i] = procesador[i].replace("\xa0"," ")
    procesador[i] = procesador[i].replace("Procesador:","")
    procesador[i] = procesador[i].lstrip()

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ","")
    precios[i] = precios[i].replace("$","")
    precios[i] = precios[i].replace(".","")

for i in range(len(marca)):
    nombres.append(marca[i] + " " + modelo[i])

for i in range(len(codigos)):
    codigos[i] = codigos[i].replace("ID ","")

#Obtención de fecha y tienda
x = datetime.today().strftime('%Y-%m-%d')
tienda = "PC FACTORY"



In [ ]:
#Ingreso de valores extraidos de Pc Factory 
for i in range(len(nombres)):
    df = df.append({'Nombre': nombres[i],
                    'Codigo':codigos[i],
                    'Modelo':modelo[i] ,
                    'Almacenamiento':almacenamiento[i],
                    'Procesador':procesador[i], 
                    'Memoria Ram':ram[i], 
                    "Precio":precios[i], 
                    'Fecha':x, 
                    'Tienda':tienda , 
                    'Url': url_notebooks[i]},ignore_index=True)

# Extracción de datos en Falabella

In [ ]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,12):
    url = "https://www.falabella.com/falabella-cl/category/cat70057/Notebooks?page="+str(i)
    url_base.append(url)


##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="jsx-4001457643 search-results-list"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)

#Creación de listas auxiliares
marca = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigo = []
ram = []
precios = []

#Recorrido por cada uno de los computadores del sitio
for i in url_notebooks:
    try: 
        nombre = []
        valor = []
        driver.get(i)
        sleep(2)
        try:
            boton = driver.find_element_by_xpath('//button[@class="jsx-3459521287 mkp-swatchButton mkp-swatchButton-collapseButton"]').click()
        except:
            try:
                boton = driver.find_element_by_xpath('//button[@class="jsx-3459521287 mkp-swatchButton mkp-swatchButton-collapseButton"]').click()
            except:
                boton = driver.find_element_by_xpath('//button[@class="jsx-3459521287 mkp-swatchButton mkp-swatchButton-collapseButton"]').click()
        sleep(random.uniform(4,5))
        nombre_aux = driver.find_element_by_xpath('//div[@class="jsx-3686231685 product-name fa--product-name"]').text
        elementos = driver.find_elements_by_xpath('//tr[@class="jsx-428502957"]')
        #Obtención de precio de los productos  
        try:
            precio = driver.find_element_by_xpath('//span[@class="copy12 primary  jsx-340449923 normal    "]').text       
         
        except:
            try:
                precio = driver.find_element_by_xpath('//span[@class="copy10 primary  jsx-340449923 normal    "]').text
            except:
                try:
                    precio = driver.find_element_by_xpath('//h2[@class="jsx-2034229452 title6 mkp-secondary    "]').text
                except:
                    try:
                        precio = driver.find_element_by_xpath('//span[@class="copy12 primary  jsx-2612542277 normal     "]').text
                    except:
                        precio = driver.find_element_by_xpath('//span[@class="copy10 primary  jsx-2612542277 normal     "]').text
        
        #Obtención de codigo de los productos   
        codigos = driver.find_element_by_xpath('//span[@class="jsx-3408573263"]').text
        
        #Recorrido de los elementos dentro de cada pagina
        for j in elementos:
            
            nombres = j.find_element_by_xpath('.//td[@class="jsx-428502957 property-name"]').text
            valores = j.find_element_by_xpath('.//td[@class="jsx-428502957 property-value"]').text
            nombre.append(nombres)
            valor.append(valores)
        
        #Filtro de valores en relación al elemento buscado
        if ("Nombre comercial" in nombre)==True:
            index_modelo = nombre.index("Nombre comercial")
            modelo.append(valor[index_modelo].upper())
        
        if ("Nombre comercial" in nombre)==False:
            modelo.append("Modelo no encontrado")
        
        if("Procesador" in nombre)==True:
            index_procesador = nombre.index("Procesador")
            procesador.append(valor[index_procesador].upper())
            
        if("Procesador" in nombre)==False:
            procesador.append("Procesador no encontrado")
            
        if("Unidad de estado sólido SSD" in nombre)==True:
            index_almacenamiento = nombre.index("Unidad de estado sólido SSD")
            almacenamiento.append(valor[index_almacenamiento].upper())
            
        if("Unidad de estado sólido SSD" in nombre)==False:
            almacenamiento.append("Almacenamiento no encontrado")
            
        if("Memoria RAM" in nombre)==True:
            index_ram= nombre.index("Memoria RAM")
            ram.append(valor[index_ram].upper())
            
        if("Memoria RAM" in nombre)==False:
            ram.append("Memoria Ram no encontrada") 
            
        
        precios.append(precio)
        codigo.append(codigos)
        nombre_equipo.append(nombre_aux)

    except:
        y="Producto no disponible"
        
#Eliminación de carácteres especiales
for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")

for i in range(len(codigo)):
    codigo[i] = codigo[i].replace("Código del producto: ","")


#Obtención de fecha y tienda
x = datetime.today().strftime('%Y-%m-%d')
tienda = "Falabella"


In [ ]:
#Ingreso de datos obtenidos en Falabella dentro del dataset 
for i in range(len(nombre_equipo)):
    df = df.append({'Nombre': nombre_equipo[i],
                    'Codigo':codigo[i],
                    'Modelo':modelo[i] ,
                    'Almacenamiento':almacenamiento[i],
                    'Procesador':procesador[i], 
                    'Memoria Ram':ram[i],                     
                    "Precio":precios[i],
                    "Fecha":x, 
                    'Tienda':tienda , 
                    'Url': url_notebooks[i] },ignore_index=True)

# Extracción de datos en HP

In [ ]:
#Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,13):
    url = "https://www.hp.com/cl-es/shop/notebooks/notebooks-intel.html?gclsrc=aw.ds&p="+str(i)
    url_base.append(url)

##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//li[@class="item product product-item"]')
    for i in notebook:
        try:
            link = i.find_element_by_tag_name('a').get_attribute('href')
        except:
            x = 2
        url_notebooks.append(link)

##Listas que almacenaran datos encontrados dentro del sitio web
marca = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigo = []
ram = []
precios = []

#Recorrido por todos los equipos dentro del sitio web
for i in url_notebooks:
    nombres = []
    valores = []
    sleep(2)
    driver.get(i)
    try:
        boton = driver.find_element_by_xpath('//div[@class="data item title" and @id="tab-label-additional"]').click()
    except:
        try:

            boton = driver.find_element_by_xpath('//a[@class="data switch" and @data-toggle="trigger"').click()
        except:
            x = "boton no encontrado"
    sleep(5)
    
    #Obtención de nombre del producto y precio de cada producto
    try:
        name = driver.find_element_by_xpath('//span[@class="base" and @data-ui-id="page-title-wrapper"]').text
    except:
        name = "Nombre no encontrado"
    try:
        price = driver.find_element_by_xpath('//span[@class="price"]').text
    except:
        price = "Precio no encontrado"
    
    try:
        code = driver.find_element_by_xpath('//div[@class="value " and @itemprop="sku"]').text
    except:
        code = "Codigo no encontrado"
    bloque = driver.find_elements_by_xpath('.//tr[@class="item"]')
    for j in bloque:
        nombre = j.find_element_by_xpath('.//th[@class="col label"]').text
        valor = j.find_element_by_xpath('.//td[@class="col data"]').text
        nombres.append(nombre)
        valores.append(valor)
        
    #Obtención de caracteristicas        
    if ("Procesador" in nombres)==True:
        index_procesador = nombres.index("Procesador")
        procesador.append(valores[index_procesador])
        
    elif ("Procesador" in nombres)==False:
        procesador.append("Procesador no especificado")
        
    if ("Memoria, estándar" in nombres)==True:
        index_ram = nombres.index("Memoria, estándar")
        ram.append(valores[index_ram])
        
    elif ("Memoria, estándar" in nombres)==False:
        ram.append("Memoria ram no especificada")
        
    if("Descripción del disco duro" in nombres)==True:
        index_almacenamiento = nombres.index("Descripción del disco duro")
        almacenamiento.append(valores[index_almacenamiento])
    
    elif("Descripción del disco duro" in nombres)==False:
        almacenamiento.append("Almacenamiento no especificado")  
    
    nombre_equipo.append(name)
    precios.append(price)
    codigo.append(code)

#Eliminación de caracteres especiales
for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")
    if(precios[i]==""):
        precios[i] = "Precio no especificado"
        
for i in range(len(nombre_equipo)):
    modelo.append(nombre_equipo[i].replace("Notebook HP", ""))
    

In [ ]:
#Llenado de cada uno de los campos
x = datetime.today().strftime('%Y-%m-%d')
tienda = "HP"
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i], 
                    'Codigo':codigo[i], 
                    'Modelo':modelo[i],
                    'Almacenamiento':almacenamiento[i],
                    'Procesador':procesador[i], 
                    'Memoria Ram':ram[i], 
                    'Precio':precios[i], 
                    "Fecha":x, 
                    'Tienda':tienda , 
                    'Url': url_notebooks[i]},ignore_index=True)

# Extracción de datos en Ripley


In [ ]:
#Driver auxiliar
driver = webdriver.Chrome(ChromeDriverManager().install())

##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,10):
    url = "https://simple.ripley.cl/tecno/computacion/notebooks?page="+str(i)
    url_base.append(url)

##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="catalog-product-item catalog-product-item__container col-xs-6 col-sm-6 col-md-4 col-lg-4"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)

##Obtención de datos dentro de cada una de las páginas del sitio web, almacenados en listas auxiliares
nombre_equipo = []
precios = []
almacenamiento = []
procesador = []
ram = []
modelo = []
codigos = []

#Recorrido de todos los equipos dentro del sitio
for i in url_notebooks:
    nombres = []
    valores = []        
    driver.get(i)
    sleep(5)
    try:
        boton = driver.find_element_by_xpath('//div[@aria-controls="panel-Especificaciones"]').click()
    except:
        boton = "..."
    sleep(2)
    try:
        ver = driver.find_element_by_xpath('//button[@class="btn-loading  rpl-show-more btn-outline-grey "]').click()
    except:
        x=2
    bloque = driver.find_elements_by_xpath('//tr')
    bloque_nombre = driver.find_elements_by_xpath('//section[@class="product-header hidden-xs"]')
    try:
        precio = driver.find_element_by_xpath('//dt[@class="product-price"]')
    except:
        try:
            precio = driver.find_element_by_xpath('//dt[@class="product-price product-price__line-thru"]')
        except:
            precio = "Precio no encontrado"
            
    #Obtención de caracteristicas
    for i in bloque_nombre:
        nombre = i.find_element_by_xpath('.//h1')
        codigo = i.find_element_by_xpath('.//span[@class="sku sku-value"]')
    for i in bloque:
        elementos = i.find_elements_by_xpath('.//td')
        for j in range(len(elementos)):
            if (j==0):
                nombres.append(elementos[j].text)
            if (j==1):
                valores.append(elementos[j].text)
                
    #Filtrado de elementos por nombre
    if ("Tipo Procesador" in nombres)==True:
        index_procesador = nombres.index("Tipo Procesador")
        procesador.append(valores[index_procesador])

    if ("Tipo Procesador" in nombres)==False:
        procesador.append("Procesador no especificado")

    if ("Memoria Ram" in nombres)==True:
        index_ram = nombres.index("Memoria Ram")        
        ram.append(valores[index_ram])
        
    if ("Memoria Ram" in nombres)==False:
        ram.append("Memoria Ram no especificada")
        
    if ("Capacidad Disco Duro" in nombres)==True:
        idex_almacenamiento = nombres.index("Capacidad Disco Duro")
        almacenamiento.append(valores[idex_almacenamiento])
        
    if ("Capacidad Disco Duro" in nombres)==False:
        almacenamiento.append("Almacenamiento no especificado")
        
    if ("Modelo" in nombres)==True:
        index_modelo = nombres.index("Modelo")
        modelo.append(valores[index_modelo])
        
    if ("Modelo" in nombres)==False:
        modelo.append("Modelo no especificado")
        
    try:
        codigos.append(codigo.text)
    except:
        codigos.append("Codigo no especificado")
    try:
        precios.append(precio.text)

    except: 
        precios.append(precio)
        
    try:
        nombre_equipo.append(nombre.text)
    except:
        nombre_equipo.append("Nombre no encontrado")

#Eliminación de caracteres especiales
for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")

x = datetime.today().strftime('%Y-%m-%d')
tienda = "Ripley"




====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\ramir\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [ ]:
#Inserción de datos dentro del dataset
for i in range(len(nombre_equipo)):
    df = df.append({'Nombre': nombre_equipo[i], 
                    'Codigo':codigos[i],
                    'Modelo':modelo[i],
                    'Almacenamiento':almacenamiento[i],
                    'Procesador':procesador[i], 
                    'Memoria Ram':ram[i], 
                    "Precio":precios[i], 
                    "Fecha":x, 
                    "Tienda":tienda , 
                    'Url': url_notebooks[i]},ignore_index=True)

# Exportacion de dataset a un archivo excel

In [ ]:
#Exportación de dataframe a excel
try: 
    df = df.drop(["Unnamed: 0"],axis=1)
except:
    x = 2
df.to_excel("NotebooksOficial.xlsx")


In [ ]:
#Importación de dataset exportado para trabajo auxiliar
df = pd.read_excel("NotebooksOficial.xlsx")

# Homologar valores dentro del dataset

In [ ]:
#Almacenamiento de datos en listas auxiliares
ram = df["Memoria Ram"]
codigo = df["Codigo"]
fecha = df["Fecha"]
precio = df["Precio"]
almacenamiento = df["Almacenamiento"]
nombre = df["Nombre"]
modelo = df["Modelo"]
procesador = df["Procesador"]
tienda = df["Tienda"]
tProcesador = []
url = df["Url"]
mPros = []
marca = []

In [ ]:
#Poner el nombre del equipo en mayuscula

for i in range(len(nombre)):
    nombre[i] = nombre[i].upper()



<ipython-input-19-dfebd5862f60>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nombre[i] = nombre[i].upper()


# Homologar valores de memoria ram

In [ ]:
#Memoria Ram
for i in range(len(ram)):
    try: 
        if(math.isnan(ram[i])):
            ram[i] = "NONE"
            ram[i] = ram[i].upper()
            ram[i] = ram[i].replace(" ", "")
        else:
            ram[i] = ram[i]
            ram[i] = ram[i].upper()
            ram[i] = ram[i].replace(" ", "")
            
    except:        
        ram[i] = ram[i]
        ram[i] = ram[i].upper()
        ram[i] = ram[i].replace(" ", "")
    
    
for i in range(len(ram)):
    if(("4 GB" in ram[i])==True or ("4GB" in ram[i])==True):
        ram[i] = "4 GB"

    elif(("12 GB" in ram[i])==True or ("12GB" in ram[i])==True):
        ram[i] = "12 GB"
        
    elif(("2 GB" in ram[i])==True or ("2GB" in ram[i])==True):
        ram[i] = "12 GB" 
        
    elif(("8 GB" in ram[i])==True or ("8GB" in ram[i])==True):
        ram[i] = "8 GB"
    
    elif(("16 GB" in ram[i])==True or ("16GB" in ram[i])==True):
        ram[i] = "16 GB"
        
    elif(("6 GB" in ram[i])==True or ("6GB" in ram[i])==True):
        ram[i] = "6 GB"
        
    elif(("8 MB" in ram[i])==True or ("8MB" in ram[i])==True):
        ram[i] = "8 8GB"
        
    elif(("4 MB" in ram[i])==True or ("4MB" in ram[i])==True):
        ram[i] = "4 GB"
        
    else:
        ram[i] = "NONE"

<ipython-input-20-dc1e2610582d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ram[i] = ram[i]
<ipython-input-20-dc1e2610582d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ram[i] = ram[i].upper()
<ipython-input-20-dc1e2610582d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ram[i] = ram[i].replace(" ", "")
<ipython-input-20-dc1e2610582d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

# Segunda homologación de ram

In [ ]:
for i in range(len(nombre)):    
    if (ram[i] == "NONE"):
        if(("4 GB" in nombre[i])==True or ("4GB" in nombre[i])==True):
            ram[i] = "4 GB"

        elif(("12 GB" in nombre[i])==True or ("12GB" in nombre[i])==True):
            ram[i] = "12 GB"
            
        elif(("2 GB" in nombre[i])==True or ("2GB" in nombre[i])==True):
            ram[i] = "12 GB" 
            
        elif(("8 GB" in nombre[i])==True or ("8GB" in nombre[i])==True):
            ram[i] = "8 GB"

        elif(("16 GB" in nombre[i])==True or ("16GB" in nombre[i])==True):
            ram[i] = "16 GB"
            
        elif(("6 GB" in nombre[i])==True or ("6GB" in nombre[i])==True):
            ram[i] = "6 GB"
            
        elif(("8 MB" in nombre[i])==True or ("8MB" in nombre[i])==True):
            ram[i] = "8 8GB"
            
        elif(("4 MB" in nombre[i])==True or ("4MB" in nombre[i])==True):
            ram[i] = "4 GB"
            
        else:
            ram[i] = "NONE"
        

<ipython-input-21-1853531832f0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ram[i] = "4 GB"
<ipython-input-21-1853531832f0>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ram[i] = "NONE"
<ipython-input-21-1853531832f0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ram[i] = "12 GB"
<ipython-input-21-1853531832f0>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in

# Homologar valores de almacenamiento

In [ ]:
#Almacenamiento

for i in range(len(almacenamiento)):
    try: 
        if(math.isnan(almacenamiento[i])):
            almacenamiento[i] = "NONE"
            almacenamiento[i] = almacenamiento[i].upper()
            almacenamiento[i] = almacenamiento[i].replace(" ", "")
        else:
            almacenamiento[i] = almacenamiento[i]
            almacenamiento[i] = almacenamiento[i].upper()
            almacenamiento[i] = almacenamiento[i].replace(" ", "")
            
    except: 
              
        almacenamiento[i] = almacenamiento[i]
        almacenamiento[i] = almacenamiento[i].upper()
        almacenamiento[i] = almacenamiento[i].replace(" ", "")
    
for i in range(len(almacenamiento)):

    if(("1TB" in almacenamiento[i])==True and ("128GB" in almacenamiento[i])==True) :
        almacenamiento[i] = "1 TB HDD + 128 GB SSD"

    elif(("128 GB" in almacenamiento[i]==True)  or ("128GB" in almacenamiento[i])==True):
        almacenamiento[i] = "128 GB"
        
    elif (("256 GB" in almacenamiento[i]==True)  or ("256GB" in almacenamiento[i])==True):
        almacenamiento[i] = "256 GB"
        
    elif (("512 GB" in almacenamiento[i]==True)  or ("512GB" in almacenamiento[i])==True):
        almacenamiento[i] = "512 GB"
        
    elif (("1 TB" in almacenamiento[i]==True)  or ("1TB" in almacenamiento[i])==True):
        almacenamiento[i] = "1 TB"   
        
    elif (("500 GB" in almacenamiento[i]==True)  or ("500GB" in almacenamiento[i])==True):
        almacenamiento[i] = "500 GB" 
        
    elif (("240 GB" in almacenamiento[i]==True)  or ("240GB" in almacenamiento[i])==True):
        almacenamiento[i] = "240 GB" 
    
    elif (("320 GB" in almacenamiento[i]==True)  or ("320GB" in almacenamiento[i])==True):
        almacenamiento[i] = "320 GB" 
        
    elif (("32 GB" in almacenamiento[i]==True)  or ("32GB" in almacenamiento[i])==True):
        almacenamiento[i] = "32 GB" 
    
    elif (("64 GB" in almacenamiento[i]==True)  or ("64GB" in almacenamiento[i])==True):
        almacenamiento[i] = "64 GB" 
        
    elif (("1 TB" in almacenamiento[i]==True)  or ("1TB" in almacenamiento[i])==True):
        almacenamiento[i] = "1 TB"
        
    elif (("2 TB" in almacenamiento[i]==True)  or ("2TB" in almacenamiento[i])==True):
        almacenamiento[i] = "2 TB" 
        
    elif (("480 GB" in almacenamiento[i]==True)  or ("480GB" in almacenamiento[i])==True):
        almacenamiento[i] = "480 GB"
        
    else:
        almacenamiento[i] = "NONE"     

<ipython-input-22-885612e4e05d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  almacenamiento[i] = almacenamiento[i]
<ipython-input-22-885612e4e05d>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  almacenamiento[i] = almacenamiento[i].upper()
<ipython-input-22-885612e4e05d>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  almacenamiento[i] = almacenamiento[i].replace(" ", "")
<ipython-input-22-885612e4e05d>:6: SettingWithCopyWarn

# Segunda homologación del almacenamiento

In [ ]:
for i in range(len(nombre)):

    if(almacenamiento[i]=="NONE"):
        if(("1TB" in nombre[i])==True and ("128GB" in nombre[i])==True) :
            almacenamiento[i] = "1 TB HDD + 128 GB SSD"

        elif(("128 GB" in nombre[i]==True)  or ("128GB" in nombre[i])==True):
            almacenamiento[i] = "128 GB"
            
        elif (("256 GB" in nombre[i]==True)  or ("256GB" in nombre[i])==True):
            almacenamiento[i] = "256 GB"
            
        elif (("512 GB" in nombre[i]==True)  or ("512GB" in nombre[i])==True):
            almacenamiento[i] = "512 GB"
            
        elif (("1 TB" in nombre[i]==True)  or ("1TB" in nombre[i])==True):
            almacenamiento[i] = "1 TB"   
            
        elif (("500 GB" in nombre[i]==True)  or ("500GB" in nombre[i])==True):
            almacenamiento[i] = "500 GB" 
            
        elif (("240 GB" in nombre[i]==True)  or ("240GB" in nombre[i])==True):
            almacenamiento[i] = "240 GB" 

        elif (("320 GB" in nombre[i]==True)  or ("320GB" in nombre[i])==True):
            almacenamiento[i] = "320 GB" 
            
        elif (("32 GB" in nombre[i]==True)  or ("32GB" in nombre[i])==True):
            almacenamiento[i] = "32 GB" 

        elif (("64 GB" in nombre[i]==True)  or ("64GB" in nombre[i])==True):
            almacenamiento[i] = "64 GB" 
            
        elif (("1 TB" in nombre[i]==True)  or ("1TB" in nombre[i])==True):
            almacenamiento[i] = "1 TB"
            
        elif (("2 TB" in nombre[i]==True)  or ("2TB" in nombre[i])==True):
            almacenamiento[i] = "2 TB" 
            
        elif (("480 GB" in nombre[i]==True)  or ("480GB" in nombre[i])==True):
            almacenamiento[i] = "480 GB"
            
        else:
            almacenamiento[i] = "NONE" 

<ipython-input-23-7b40b2ab9e41>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  almacenamiento[i] = "500 GB"
<ipython-input-23-7b40b2ab9e41>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  almacenamiento[i] = "NONE"
<ipython-input-23-7b40b2ab9e41>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  almacenamiento[i] = "64 GB"
<ipython-input-23-7b40b2ab9e41>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

# Homologar valores del nombre del equipo para extracción de Marca

In [ ]:
#Nombre

for i in range(len(nombre)):
    nombre[i] = nombre[i].upper()

for i in range(len(modelo)):
    modelo[i] = str(modelo[i])
    modelo[i] = modelo[i].upper()
    
    
for i in range(len(nombre)):
    if("HP" in nombre[i]):
        marca.append("HP")
        
    elif ("LENOVO" in nombre[i]) or ("LENOVO" in modelo[i]):
        marca.append("LENOVO")
    
    elif ("ASUS" in nombre[i]) or ("ASUS" in modelo[i]):
        marca.append("ASUS")
        
    elif ("DELL" in nombre[i]) or ("DELL" in modelo[i]):
        marca.append("DELL")
        
    elif ("ACER" in nombre[i]) or ("ACER" in modelo[i]):
        marca.append("ACER")
        
    elif ("CHROMEBOOK" in nombre[i]) or ("CHROMEBOOK" in modelo[i]):
        marca.append("CHROMEBOOK")
        
    elif ("MACBOOK" in nombre[i]) or ("MACBOOK" in modelo[i]):
        marca.append("MACBOOK")
        
    elif ("MSI" in nombre[i]) or ("MSI" in modelo[i]):
        marca.append("MSI")
        
    elif ("IDEAPAD" in nombre[i]) or ("IDEAPAD" in modelo[i]):
        marca.append("IDEAPAD")
        
    elif ("PAVILION" in nombre[i]) or ("PAVILION" in modelo[i]):
        marca.append("PAVILION")
        
    elif ("HUAWEI" in nombre[i]) or ("HUAWEI" in modelo[i]):
        marca.append("HUAWEI")
        
    elif ("HEROBOOK" in nombre[i]) or ("HEROBOOK" in modelo[i]):
        marca.append("HEROBOOK")
        
    elif ("ALIENWARE" in nombre[i]) or ("ALENWARE" in modelo[i]):
        marca.append("ALIENWARE")
        
    elif ("MATEBOOK" in nombre[i]) or ("MATEBOOK" in modelo[i]):
        marca.append("MATEBOOK")
        
    elif ("LEGION" in nombre[i]) or ("LEGION" in modelo[i]):
        marca.append("LEGION")
        
    elif ("PACKOMEN" in nombre[i]) or ("PACKOMEN" in modelo[i]):
        marca.append("OMEN")
        
    elif ("ASPIRE" in nombre[i]) or ("ASPIRE" in modelo[i]):
        marca.append("ASPIRE")
        
    elif ("CORSAIR" in nombre[i]) or ("CORSAIR" in modelo[i]):
        marca.append("CORSAIR")
      
    elif ("GATEWAY" in nombre[i]) or ("GATEWAY" in modelo[i]):
        marca.append("GATEWAY")  
    
    elif ("GETAC" in nombre[i]) or ("GETAC" in modelo[i]):
        marca.append("GETAC")
        
    elif ("MAGICBOOK" in nombre[i]) or ("MAGICBOOK" in modelo[i]):
        marca.append("MAGICBOOK")
        
    elif ("IMAC" in nombre[i]) or ("IMAC" in modelo[i]):
        marca.append("IMAC")
        
    elif ("INSPIRON" in nombre[i]) or ("INSPIRON" in modelo[i]):
        marca.append("INSPIRON")
    
    elif ("MODERN" in nombre[i]) or ("MODERN" in modelo[i]):
        marca.append("MODERN")
        
    elif ("ALL IN ONE" in nombre[i]) or ("ALL IN ONE" in modelo[i]):
        marca.append("ALL IN ONE")
        
    elif ("YOGA" in nombre[i]) or ("YOGA" in modelo[i]):
        marca.append("YOGA")
        
    elif ("ZBOOK" in nombre[i]) or ("ZBOOK" in modelo[i]):
        marca.append("ZBOOK")
        
    elif ("KATANA" in nombre[i]) or ("KATANA" in modelo[i]):
        marca.append("KATANA")
        
    elif ("AERO" in nombre[i]) or ("AERO" in modelo[i]):
        marca.append("AERO")
        
    elif ("CHUWI" in nombre[i]) or ("CHUWI" in modelo[i]):
        marca.append("CHUWI")
        
    elif ("ROG ZEPHYRUS" in nombre[i]) or ("ROG ZEPHYRUS" in modelo[i]):
        marca.append("ROG ZEPHYRUS")
        
    elif ("BMAX" in nombre[i]) or ("BMAX" in modelo[i]):
        marca.append("BMAX")
        
    elif ("TOSHIBA" in nombre[i]) or ("TOSHIBA" in modelo[i]):
        marca.append("TOSHIBA")
        
    elif ("SURFACE" in nombre[i]) or ("SURFACE" in modelo[i]):
        marca.append("SURFACE")
        
    elif ("ZENBOOK" in nombre[i]) or ("ZENBOOK" in modelo[i]):
        marca.append("ZENBOOK")
        
    elif ("VIVOBOOK" in nombre[i]) or ("VIVOBOOK" in modelo[i]):
        marca.append("VIVOBOOK")
        
    elif ("XPS" in nombre[i]) or ("XPS" in modelo[i]):
        marca.append("XPS")
        
    elif ("XPG" in nombre[i]) or ("XPG" in modelo[i]):
        marca.append("XPG")
        
    elif ("TUF" in nombre[i]) or ("TUF" in modelo[i]):
        marca.append("TUF")
        
    elif ("THINKPAD" in nombre[i]) or ("THINKPAD" in modelo[i]):
        marca.append("THINKPAD")
        
    elif ("SWIFT" in nombre[i]) or ("SWIFT" in modelo[i]):
        marca.append("SWIFT")
        
    elif ("LATITUD" in nombre[i] or ("LATITUDE" in nombre[i]) or ("LATITUD" in modelo[i])):
        marca.append("LATITUD")
        
    elif ("EVOO" in nombre[i]) or ("EVOO" in modelo[i]):
        marca.append("EVOO")
        
    elif ("ROG" in nombre[i]) or ("ROG" in modelo[i]):
        marca.append("ROG")
        
    elif ("ELITEBOOK" in nombre[i]) or ("ELITEBOOK" in modelo[i]):
        marca.append("ELITEBOOK")
        
    elif ("THINKBOOK" in nombre[i]) or ("THINKBOOK" in modelo[i]):
        marca.append("THINKBOOK")
        
    elif ("NITRO" in nombre[i]) or ("NITRO" in modelo[i]):
        marca.append("NITRO")
        
    elif ("DYNABOOK" in nombre[i]) or ("DYNABOOK" in modelo[i]):
        marca.append("DYNABOOK")
        
    elif ("AORUS" in nombre[i]) or ("AORUS" in modelo[i]):
        marca.append("AORUS") 

    elif ("OMEN" in nombre[i]) or ("OMEN" in modelo[i]):
        marca.append("OMEN") 

    elif ("PROBOOK" in nombre[i]) or ("PROBOOK" in modelo[i]):
        marca.append("PROBOOK") 

    elif ("SPECTRE" in nombre[i]) or ("SPECTRE" in modelo[i]):
        marca.append("SPECTRE")

    elif ("EXPERTBOOK" in nombre[i]) or ("EXPERTBOOK" in modelo[i]):
        marca.append("EXPERTBOOK")
    
    elif ("CELERON" in nombre[i]) or ("CELERON" in modelo[i]):
        marca.append("CELERON")
    
    else:
        marca.append("NONE")

<ipython-input-24-c9e5bbf02afb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nombre[i] = nombre[i].upper()
<ipython-input-24-c9e5bbf02afb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modelo[i] = str(modelo[i])
<ipython-input-24-c9e5bbf02afb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modelo[i] = modelo[i].upper()


# Homologar valores del procesador

In [ ]:
#Procesador

for i in range(len(procesador)):
    try: 
        if(math.isnan(procesador[i])):
            procesador[i] = "NONE"
            procesador[i] = procesador[i].upper()
            procesador[i] = procesador[i].replace(" ", "")
        else:
            procesador[i] = procesador[i]
            procesador[i] = procesador[i].upper()
            procesador[i] = procesador[i].replace(" ", "")
            
    except: 
              
        procesador[i] = procesador[i]
        procesador[i] = procesador[i].upper()
        procesador[i] = procesador[i].replace(" ", "")

for i in range(len(procesador)):

    if("AMD" in procesador[i]):
        mPros.append("AMD")

    elif("INTEL" in procesador[i]):
        mPros.append("INTEL")

    elif ("APPLE" in procesador[i]):
        mPros.append("APPLE")

    elif("MEDIATEK" in procesador[i]):
        mPros.append("MEDIATEK")

    elif("RYZEN" in procesador[i]):
        mPros.append("AMD")

    elif(("I3" in procesador[i]) or ("I5" in procesador[i]) or ("I7" in procesador[i])):
        mPros.append("INTEL")

    elif("PENTIUM" in procesador[i]):
        mPros.append("INTEL")
        
    elif("CELERON" in procesador[i]):
        mPros.append("INTEL")
    
    else:
        mPros.append("NONE")

<ipython-input-25-188c84f7962b>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  procesador[i] = procesador[i]
<ipython-input-25-188c84f7962b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  procesador[i] = procesador[i].upper()
<ipython-input-25-188c84f7962b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  procesador[i] = procesador[i].replace(" ", "")
<ipython-input-25-188c84f7962b>:6: SettingWithCopyWarning: 
A value is trying 

# Segunda homologación de procesador

In [ ]:
#Segunda homologación del procesador
for i in range(len(nombre)):
    if(mPros[i]=="NONE"):
        if("AMD" in nombre[i]):
            mPros[i] = "AMD"

        elif("INTEL" in nombre[i]):
            mPros[i] ="INTEL"

        elif ("APPLE" in nombre[i]):
            mPros[i] ="APPLE"

        elif("MEDIATEK" in nombre[i]):
            mPros[i] ="MEDIATEK"

        elif("RYZEN" in nombre[i]):
            mPros[i] ="AMD"

        elif(("I3" in nombre[i]) or ("I5" in nombre[i]) or ("I7" in nombre[i])):
            mPros[i] ="INTEL"

        elif("PENTIUM" in nombre[i]):
            mPros[i] ="INTEL"
            
        elif("CELERON" in nombre[i]):
            mPros[i] ="INTEL"
        
        else:
            mPros[i] ="NONE"

In [ ]:
#Modelo de precios

for i in range(len(precio)):
       
    try:
        x = int(precio[i])
        
    except:
        precio[i] = 0
        

df['Precio'] = df['Precio'].apply(pd.to_numeric)

<ipython-input-27-41ce129c4b16>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precio[i] = 0


# Homologar precios

In [ ]:
#Modelo de precios

for i in range(len(precio)):
    
    if(precio[i]=="Precionoencontrado" or precio[i]=="Precio no especificado" or precio[i]=="¡Quémal!Justoseagotó" or precio[i]=="Nodisponible"):        
        precio[i] = 0

df['Precio'] = df['Precio'].apply(pd.to_numeric)

# Homologar modelos del procesador

In [ ]:
#Modelo de procesador

for i in range(len(procesador)):

    if(procesador[i] == "PROCESADOR NO ENCONTRADO") or (procesador[i] == "PROCESADOR NO ESPECIFICADO") or (procesador[i]=="NO APLICA") or(procesador[i]=="PROCESADORNOESPECIFICADO"):
        procesador[i] = "NONE"
    
    if(mPros[i]=="AMD"):
        if("ATHLON" in procesador[i]):
            tProcesador.append("ATHLON")
            
        elif(("RYZEN" in procesador[i]) and ("5" in procesador[i])):
            tProcesador.append("RYZEN 5")
            
        elif(("RYZEN" in procesador[i]) and ("3" in procesador[i])):
            tProcesador.append("RYZEN 3")
            
        elif(("RYZEN" in procesador[i]) and ("7" in procesador[i])):
            tProcesador.append("RYZEN 7")
            
        elif(("RYZEN" in procesador[i]) and ("9" in procesador[i])):
            tProcesador.append("RYZEN 9")
            
        elif(("RYZEN" in procesador[i]) and ("R3" in procesador[i])):
            tProcesador.append("RYZEN R3")
            
        elif(("RYZEN" in procesador[i]) and ("R5" in procesador[i])):
            tProcesador.append("RYZEN R5")
            
        elif(("RYZEN" in procesador[i]) and ("R7" in procesador[i])):
            tProcesador.append("RYZEN R7")
            
        elif("RYZEN" in procesador[i]):
            tProcesador.append("RYZEN")
            
        elif("3000 SERIES" in procesador[i]):
            tProcesador.append("3000 SERIES")
            
        elif("A10" in procesador[i]):
            tProcesador.append("A10")
            
        elif("A12" in procesador[i]):
            tProcesador.append("A12")
            
        elif("A4" in procesador[i]):
            tProcesador.append("A4")
            
        elif("E2" in procesador[i]):
            tProcesador.append("E2")
            
        elif("E-SERIES" in procesador[i]):
            tProcesador.append("E-SERIES")
            
        elif("SERIE A" in procesador[i]):
            tProcesador.append("SERIE A")
            
        else:
            tProcesador.append(procesador[i])
            
    elif(mPros[i]=="APPLE"):
        if("M1" in procesador[i]):
            tProcesador.append("M1")
            
        else:
            tProcesador.append(procesador[i])
            
    elif(mPros[i]=="INTEL"):
        if("CELERON" in procesador[i]):
            tProcesador.append("CELERON")
            
        elif("PENTIUM" in procesador[i]  and "GOLD" in procesador[i]):
            tProcesador.append("PENTIUM GOLD")

        elif("PENTIUM" in procesador[i]  and "SILVER" in procesador[i]):
            tProcesador.append("PENTIUM SILVER")
            
        elif("CORE" in procesador[i]) and ("I3" in procesador[i]):
            tProcesador.append("CORE I3")
            
        elif("CORE" in procesador[i]) and ("3" in procesador[i]):
            tProcesador.append("CORE I3")
            
        elif("CORE" in procesador[i]) and ("I5" in procesador[i]):
            tProcesador.append("CORE I5")
            
        elif("CORE" in procesador[i]) and ("I7" in procesador[i]):
            tProcesador.append("CORE I7")
            
        elif("CORE" in procesador[i]) and ("I9" in procesador[i]):
            tProcesador.append("CORE I9")
            
        elif("CORE" in procesador[i]) and ("M" in procesador[i]):
            tProcesador.append("CORE M")
            
        elif("I5" in procesador[i]):
            tProcesador.append("CORE I5")
            
        elif("I3" in procesador[i]):
            tProcesador.append("CORE I3")
            
    
        else:
            tProcesador.append("NONE")
    else:
        tProcesador.append("NONE")

<ipython-input-29-ffcddf06142a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  procesador[i] = "NONE"


# Segunda homologación del modelo de procesador

In [ ]:
#Modelo de procesador

for i in range(len(nombre)):
    if(tProcesador[i] == "NONE"):    
        if(mPros[i]=="AMD"):
            if("ATHLON" in nombre[i]):
                tProcesador[i] = "ATHLON"
                
            elif(("RYZEN" in nombre[i]) and ("5" in nombre[i])):
                tProcesador[i] ="RYZEN 5"
                
            elif(("RYZEN" in nombre[i]) and ("3" in nombre[i])):
                tProcesador[i] ="RYZEN 3"
                
            elif(("RYZEN" in nombre[i]) and ("7" in nombre[i])):
                tProcesador[i] ="RYZEN 7"
                
            elif(("RYZEN" in nombre[i]) and ("9" in nombre[i])):
                tProcesador[i] ="RYZEN 9"
                
            elif(("RYZEN" in nombre[i]) and ("R3" in nombre[i])):
                tProcesador[i] ="RYZEN R3"
                
            elif(("RYZEN" in nombre[i]) and ("R5" in nombre[i])):
                tProcesador[i] ="RYZEN R5"
                
            elif(("RYZEN" in nombre[i]) and ("R7" in nombre[i])):
                tProcesador[i] ="RYZEN R7"
                
            elif("RYZEN" in nombre[i]):
                tProcesador[i] ="RYZEN"
                
            elif("3000 SERIES" in nombre[i]):
                tProcesador[i] ="3000 SERIES"
                
            elif("A10" in nombre[i]):
                tProcesador[i] ="A10"
                
            elif("A12" in nombre[i]):
                tProcesador[i] ="A12"
                
            elif("A4" in nombre[i]):
                tProcesador[i] ="A4"
                
            elif("E2" in nombre[i]):
                tProcesador[i] ="E2"
                
            elif("E-SERIES" in nombre[i]):
                tProcesador[i] ="E-SERIES"
                
            elif("SERIE A" in nombre[i]):
                tProcesador[i] ="SERIE A"
                
            else:
                tProcesador[i] ="NONE"
                
        elif(mPros[i]=="APPLE"):
            if("M1" in nombre[i]):
                tProcesador[i] ="M1"
                
            else:
                tProcesador[i] = "NONE"
                
        elif(mPros[i]=="INTEL"):
            if("CELERON" in nombre[i]):
                tProcesador[i] ="CELERON"
                
            elif("PENTIUM" in nombre[i]  and "GOLD" in nombre[i]):
                tProcesador[i] ="PENTIUM GOLD"

            elif("PENTIUM" in nombre[i]  and "SILVER" in nombre[i]):
                tProcesador[i] ="PENTIUM SILVER"
                
            elif("CORE" in nombre[i]) and ("I3" in nombre[i]):
                tProcesador[i] ="CORE I3"
                
            elif("CORE" in nombre[i]) and ("3" in nombre[i]):
                tProcesador[i] ="CORE I3"
                
            elif("CORE" in nombre[i]) and ("I5" in nombre[i]):
                tProcesador[i] ="CORE I5"
                
            elif("CORE" in nombre[i]) and ("I7" in nombre[i]):
                tProcesador[i] ="CORE I7"
                
            elif("CORE" in nombre[i]) and ("I9" in nombre[i]):
                tProcesador[i] ="CORE I9"
                
            elif("CORE" in nombre[i]) and ("M" in nombre[i]):
                tProcesador[i] ="CORE M"
                
            elif("I5" in nombre[i]):
                tProcesador[i] ="CORE I5"
                
            elif("I3" in nombre[i]):
                tProcesador[i] ="CORE I3"
                
        
            else:
                tProcesador[i] ="NONE"
        else:
           tProcesador[i] ="NONE"

# Homologar valores para codigo de producto

In [ ]:
#Homologar codigo de producto
for i in range(len(codigo)):
    codigo[i] = str(codigo[i])

    
for i in range(len(tienda)):
    try:
        if(tienda[i]=="Falabella"):       
            codigo[i] = codigo[i].replace("sku","")

        elif (tienda[i]=="Ripley"):    
            codigo[i] = codigo[i].replace("MPM000", "")
            codigo[i] = codigo[i].replace("P", "")

        if(codigo[i]=="2000000000000" or codigo[i]==2000000000000 or codigo[i]=="2E+12" or codigo[i]==2E+12):
            codigo[i] = "NONE"                 

        elif(tienda[i]=="Hites"):
            codigo[i] =str(codigo[i])
            codigo[i] = codigo[i].replace("SKU: ","")

        elif(tienda[i]=="HP"):
            codigo[i] = str(codigo[i])

        elif(tienda[i]=="PC FACTORY"):
            codigo[i] = str(codigo[i])
    except:
        x=2

    

<ipython-input-31-a24e13de3bbc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codigo[i] = str(codigo[i])
<ipython-input-31-a24e13de3bbc>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codigo[i] =str(codigo[i])
<ipython-input-31-a24e13de3bbc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  codigo[i] = codigo[i].replace("SKU: ","")
<ipython-input-31-a24e13de3bbc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy o

# Homologar valores del modelo del equipo

In [ ]:
#Modificar modelo de equipo

for i in range(len(modelo)):
    if(marca[i] == modelo[i]):
        modelo[i] = modelo[i]


    elif(marca[i] in modelo[i]):
        aux = marca[i]+" "
        modelo[i] = modelo[i].replace(aux, "")
        modelo[i] = modelo[i].replace("NOTEBOOK", "")

    else:
        modelo[i] = modelo[i]   

    if(modelo[i]=="MODELO NO ENCONTRADO"):
        modelo[i] = "NONE"

<ipython-input-32-21cfbbb25438>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modelo[i] = modelo[i].replace(aux, "")
<ipython-input-32-21cfbbb25438>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modelo[i] = modelo[i].replace("NOTEBOOK", "")
<ipython-input-32-21cfbbb25438>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modelo[i] = modelo[i]
<ipython-input-32-21cfbbb25438>:17: SettingWithCopyWarning: 
A value is trying to be s

In [ ]:
for i in range(len(tienda)):
    tienda[i] = tienda[i].upper()

<ipython-input-33-13de21acc7e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tienda[i] = tienda[i].upper()


In [ ]:
#Creación de nuevo dataset para almacenamiento de datos
aux = pd.DataFrame(columns=["NOMBRE", 
                            "MARCA", 
                            "RAM", 
                            "CODIGO",
                            "ALMACENAMIENTO", 
                            "PROCESADOR",
                            "MPROCESADOR",
                            "TPROCESADOR" ,
                            "PRECIO", 
                            "MODELO", 
                            "FECHA", 
                            "TIENDA", 
                            "URL"])

for i in range(len(marca)):
    aux = aux.append({"NOMBRE": nombre[i], 
                      "MARCA":marca[i], 
                      "RAM":ram[i], 
                      "CODIGO":codigo[i],
                      "ALMACENAMIENTO":almacenamiento[i], 
                      "PROCESADOR": procesador[i],
                      "MPROCESADOR":mPros[i] ,
                      'TPROCESADOR':tProcesador[i],
                      "PRECIO":precio[i], 
                      "MODELO":modelo[i], 
                      "FECHA":fecha[i], 
                      "TIENDA":tienda[i], 
                      "URL":url[i] }, ignore_index= True)

aux["PRECIO"]=aux["PRECIO"].apply(pd.to_numeric)


# Limpieza de filas del dataset con registros vacios

In [ ]:
#Limpiar filas

for i in range(len(aux)):
    if(aux["PRECIO"][i]==0 
       or aux["NOMBRE"][i] == "NOMBRE NO ENCONTRADO"
       or aux["MODELO"][i]=="NONE" 
       or aux["MODELO"][i]=="NO APLICA" 
       or aux["MODELO"][i]=="MODELO NO ESPECIFICADO" 
       or aux["CODIGO"][i]=="NONE"
       or aux["CODIGO"][i] == "CODIGO NO ESPECIFICADO" 
       or aux["RAM"][i] == "NONE" 
       or aux["MPROCESADOR"][i] == "NONE" 
       or aux["TPROCESADOR"][i] == "NONE" 
       or aux["TPROCESADOR"][i] == "PROCESADORNOESPECIFICADO" 
       or aux["TPROCESADOR"][i] == "PROCESADORNOENCONTRADO" 
       or aux["ALMACENAMIENTO"][i]=="NONE"):
        aux = aux.drop(i)

#Limpiar fecha

aux["FECHA"] = pd.to_datetime(aux.FECHA)


In [ ]:
aux = aux.reset_index()
try:
    aux = aux.drop(columns={"level_0", "index"}, axis=1)
except:
    y=2

In [ ]:
#Definición del dataset auxOficial
auxOficial = pd.DataFrame(columns=["ID_SKU",
                                   "CODIGOPROD",
                                   "NOMBRE",
                                   "MODELO", 
                                   "MARCA", 
                                   "RAM", 
                                   "ALMACENAMIENTO", 
                                   "MPROCESADOR", 
                                   "TPROCESADOR", 
                                   "PRECIO", 
                                   "FECHA",
                                   "TIENDA", 
                                   "URL"])

In [ ]:
#Creación de listas auxiliares con las columnas del dataframe
marca = []
nombre = []
codigo = []
ram = []
almacenamiento = []
mprocesador = []
modelo = []
tprocesador = []
tienda = []
precio = []
modelo = []
fecha = []
id = []
url = []

In [ ]:
#Llenado del dataset Aux 
for i in range(len(aux)):
    nombre.append(aux["NOMBRE"][i])
    marca.append(str(aux["MARCA"][i]))
    codigo.append(str(aux["CODIGO"][i]))
    ram.append(str(aux["RAM"][i]))
    almacenamiento.append(str(aux["ALMACENAMIENTO"][i]))
    mprocesador.append(str(aux["MPROCESADOR"][i]))
    modelo.append(str(aux["MODELO"][i]))
    tprocesador.append(str(aux["TPROCESADOR"][i]))
    tienda.append(str(aux["TIENDA"][i]))
    precio.append(str(aux["PRECIO"][i]))
    fecha.append(aux["FECHA"][i])
    url.append(aux["URL"][i])
    id.append(0)

In [ ]:
# Primera validación de valores iguales dentro del dataset
i=0
j=0
aux2 = 1
for i in range(len(aux)):
    cont = 0
    nombre2 = nombre[i]
    marca2 = marca[i]
    ram2 = ram[i]
    alma = almacenamiento[i]
    mprocesador2 = mprocesador[i]
    tprocesador2 = tprocesador[i]
    modelo2 = modelo[i] 
    codigo2 = codigo[i]
    url2 = url[i]   
    for j in range(len(aux)):
        if((marca2 == marca[j]) 
        and (ram2 == ram[j]) 
        and (alma == almacenamiento[j]) 
        and (mprocesador2 == mprocesador[j]) 
        and (tprocesador2 == tprocesador[j])
        and (modelo2 == modelo[j])
        and (id[j]==0)):
            id[j] = aux2
            nombre[j] = nombre2
            cont = cont + 1
            
        elif (id[j]!=0):
            id[j] = id[j]

    if (cont!=0):    
        aux2 = aux2+1  
        

In [ ]:
#Llenado del dataset "auxOficial"
for i in range(len(marca)):
    auxOficial = auxOficial.append({"ID_SKU":id[i], 
                      "NOMBRE":nombre[i],
                      "MARCA":marca[i],
                      "MODELO":modelo[i], 
                      "CODIGOPROD":codigo[i],
                      "RAM":ram[i], 
                      "ALMACENAMIENTO":almacenamiento[i], 
                      "MPROCESADOR":mprocesador[i], 
                      "TPROCESADOR":tprocesador[i],
                      "FECHA":fecha[i], 
                      "PRECIO":precio[i],
                      "URL":url[i], 
                      "TIENDA":tienda[i]}, ignore_index=True)

In [ ]:
#Creación del dataFrame id_sku para el almacenamiento de la información homologada 
id_sku = pd.DataFrame(columns=["ID", 
                               "CODIGO", 
                               "NOMBRE", 
                               "MODELO",
                               "MARCA", 
                               "RAM", 
                               "ALMACENAMIENTO", 
                               "MPROCESADOR", 
                               "TPROCESADOR"])

In [ ]:
#Definición de listas auxiliares que contienen todos los datos presentes de las columnas "auxOficial"
id = auxOficial["ID_SKU"]
cod = auxOficial["CODIGOPROD"]
mod = auxOficial["MODELO"]
nom = auxOficial["NOMBRE"]
mar = auxOficial["MARCA"]
ram = auxOficial["RAM"]
alm = auxOficial["ALMACENAMIENTO"]
mpro = auxOficial["MPROCESADOR"]
tpro = auxOficial["TPROCESADOR"]
pre = auxOficial["PRECIO"]
fec = auxOficial["FECHA"]
url = auxOficial["URL"]
tien = auxOficial["TIENDA"]


In [ ]:
#Creación de listas auxiliares para obtener los elementos iguales dentro del dataset
id_sku2 = []
codigo = []
nombre = []
modelo = []
marca = []
ram2 = []
almacenamiento = []
mprocesador = []
tprocesador = []
for i in range(len(set(id))):
    pos = 0
    for j in range(len(id)):
        if (i+1 == id[j]):
            pos = j
    #Llenado de listas auxiliares
    id_sku2.append(id[pos])
    codigo.append(cod[pos])
    modelo.append(mod[pos])
    nombre.append(nom[pos])
    marca.append(mar[pos])
    ram2.append(ram[pos])
    almacenamiento.append(alm[pos])
    mprocesador.append(mpro[pos])
    tprocesador.append(tpro[pos])

In [ ]:
# Llenado de dataset id_sku
for i in range(len(id_sku2)):
    id_sku = id_sku.append({"ID":id_sku2[i], 
                            "CODIGO":codigo[i], 
                            "MODELO":modelo[i],
                            "NOMBRE":nombre[i], 
                            "MARCA":marca[i], 
                            "RAM":ram2[i], 
                            "ALMACENAMIENTO":almacenamiento[i], 
                            "MPROCESADOR":mprocesador[i], 
                            "TPROCESADOR":tprocesador[i]}, 
                           ignore_index=True)

In [ ]:
# Definición de dataFrame "dataset", la cual contiene las columnas restantes del dataFrame "auxOficial"
dataset = auxOficial.drop(columns={"CODIGOPROD",
                                   "NOMBRE",
                                   "MODELO",	
                                   "MARCA", 
                                   "RAM", 
                                   "ALMACENAMIENTO",	
                                   "MPROCESADOR",	
                                   "TPROCESADOR", 
                                   "URL"}, 
                          axis=1)
dataset = dataset.drop_duplicates()
dataset = dataset.reset_index()
dataset = dataset.drop(columns={"index"}, axis=1)
try:    
    dataset = dataset.drop(columns={"level_0"}, axis=1)
except:
    x=2

In [ ]:
#Exportacion a xlsx de valores con el id sku de cada producto con sus caracteristicas
id_sku.to_excel("valoresSku.xlsx")
#Exportacion a xlsx de valores con el id sku, el precio, la fecha y la tienda en la que se encuentra el producto
dataset.to_excel("precioFechaPorId.xlsx")
#Exportacion a xlsx del datasetCompleto Homologado y con la limpieza de valores "basura"
auxOficial.to_excel("DatasetCompleto.xlsx")